In [1]:
import astroprov
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import SkyCoord
from astropy.table import QTable

import matplotlib.pyplot as plt
import numpy as np
import os
from regions.core import PixCoord
from regions.shapes.circle import CirclePixelRegion
import subprocess
import time
import matplotlib.text as mpl_text
import math
import pandas as pd
import pandasql as ps

In [2]:
imageRefernce = '/home/mj1e16/moleGazer/Photos/F/29018.jpg'
imageAlign =  '/home/mj1e16/moleGazer/Photos/F/79948.jpg'
# original image names

imageRefernceFits = '/home/mj1e16/moleGazer/Photos/F/fitsImages/16bit/29018InvertedAverageCropped16.fits'
imageAlignFits =  '/home/mj1e16/moleGazer/Photos/F/fitsImages/16bit/79948InvertedAverageCropped16.fits'
# processed image names


## max setting [6.444444444444445, 21.842105263157894, 'gauss_5.0_9x9.conv']

refTab = Table.read('/home/mj1e16/moleGazer/Photos/F/fitsImages/simMolePos/29018InvertedAverageCropped166.44444444444_21.8421052632_gauss_5.0_9x9.conv.cat',format='ascii.sextractor')
alignTab = Table.read('/home/mj1e16/moleGazer/Photos/F/fitsImages/simMolePos/79948InvertedAverageCropped166.44444444444_21.8421052632_gauss_5.0_9x9.conv.cat',format='ascii.sextractor')
# names of aperture photometry output tables

In [3]:
def makeDS9RegFile(sexTabList,fileNameBase,tabType,colour='red',radius='10'):
    '''
    Transforms a list of SExtractor, DAOphot, or IRAF output tables to a ds9 region file.
    
    Parameters:
        sexTabList (list): List of SExtractor output tables.
        fileNameBase (str): Naming convention for ds9 region file.
        tabType (str): Type of output file.
        colour (str): Colour of apertures created in ds9 region file.
        radius (str): Size of apertures created in ds9 region file.
        
    Returns:
        fileName (str): Name of created ds9 region file.
    
    '''
    # tabType can take values: 'dao' for DAOphot; 'sexMedian' to use RA and DEC; 'sex' for SExtractor; 'iraf' for IRAF  
    for tables in range(len(sexTabList)):
        if tabType == 'dao':
            xcoords = sexTabList[tables]['xcentroid']
            ycoords = sexTabList[tables]['ycentroid']
        elif tabType == 'sexMedian':
            ra = sexTabList[tables]['RA']
            dec = sexTabList[tables]['DEC']
        elif tabType == 'sex':
            xcoords = sexTabList[tables]['X_IMAGE']
            ycoords = sexTabList[tables]['Y_IMAGE']
        elif tabType == 'iraf':
            xcoords = sexTabList[tables]['X_POS']
            ycoords = sexTabList[tables]['Y_POS']        
        else:
            print('tab type error')
            break
        bigString = 'image\n'
        for x in range(len(xcoords)):
            bigString += 'circle({},{},'.format(xcoords[x],ycoords[x])+radius+') # color={}\n'.format(colour)
        fileName = fileNameBase + '.reg'
        with open(fileName,'w') as f:
            f.write(bigString)
            
    return fileName

In [ ]:
makeDS9RegFile([alignTab],'/home/mj1e16/moleGazer/MoleGazer/79948Regions','sex')
makeDS9RegFile([refTab],'/home/mj1e16/moleGazer/MoleGazer/29018Regions','sex')

In [4]:
# could use a re-work

def findTestSources(sources,lowx, highx, lowy, highy, imageNo):
    """
    Searches for bright objects to be used for pattern recognition within a specified region of the reference image.

    :param sources: Table of object data
    :param lowx: Lower X limit of for searching region region
    :param highx: Upper X limit
    :param lowy: Lower Y limit
    :param highy: Upper Y limit
    :param imageNo: Reference image number
    :return: Indexes in sources of (up to) the brightest 15 objects contained within the specified region
    """
    testsources = {}
    mag = []
    number = []
    #for x in range(sources[imageNo][-1][0]):
    for x in range(len(sources[imageNo])):
        x1 = sources[imageNo]['X_IMAGE'][x]
        y1 = sources[imageNo]['Y_IMAGE'][x]
        if x1 > lowx and x1 < highx and y1 > lowy and y1 < highy:
            testsources[x] = sources[imageNo][x]
            mag.append(sources[imageNo]['MAG_BEST'][x])
            number.append(x)
    Z2 = [x for _,x in sorted(zip(mag,number))]
    samplestars = Z2[0:15]
    
    #astroprov.provcall([sources,lowx, highx, lowy, highy, imageNo],[samplestars],"findTestSources_PythonCode2PythonCode_SQ_tmpl.provn","findTestSources")
    return samplestars

In [5]:
def getImageData(imageName,extension):
    '''
    Extract image data from file name.
    
    Parameters:
        imageName (str): Name of image.
        extension (int): Extension to be extracted.
        
    Returns:
        imageData (array): 2-D numpy array of image data.
    '''
    hdu = fits.open(imageName)
    imageData = hdu[extension].data
    return imageData

In [6]:
def ds9File2Lists(fileName):
    '''
    Transform objects within ds8 region file to python lists.
    
    Parameters:
        fileName (str): Name of ds9 region file.
        
    Returns:
        [xcoord,ycoord,radius] ([lst,lst,lst]: [object x positions, object y positions, object radius].
    '''
    with open(fileName,'r') as f:
        data = f.readlines()

    ycoord = []
    xcoord = []
    radius = []
    starLocations = []
    for x in data[:-1]:
        try:
            locator1 = '('
            locator2 = ','
            locator3 = ')'
            loc1 = x.index(locator1) +1
            #print(x)
            loc2 = x[loc1:].index(locator2) + loc1 +1
            loc3 = x[loc2:].index(locator2) + loc2
            loc4 = x.index(locator3)
            xcoord.append(float(x[loc1:loc2-1]))
            ycoord.append(float(x[loc2:loc3]))
            radius.append(float(x[loc3+1:loc4]))
        except:
            print('Line - {}'.format(x))
    return [xcoord,ycoord,radius]

In [7]:
def plotMolesOnly(image,moleLocFile):
    '''
    Plot mole positions on an image.
    
    Parameters:
        image (str): Image name.
        moleLocFile (str): DS9 region file contianing mole positions
    '''
    # Function doesn't return anything, just creates the image
    imdata = getImageData(image,0)
    moleLocs = ds9File2Lists(moleLocFile)
    alignlocs = moleLocs
    mask = []
    #maskData = []
    for mole in range(len(alignlocs[0])):
        centre = PixCoord(alignlocs[0][mole],alignlocs[1][mole])
        reg = CirclePixelRegion(centre,alignlocs[2][mole])
        mask.append(reg.to_mask())
        #maskData.append(mask[mole].cutout(imdata)*mask[mole].data)
    
    fig, [ax1,ax2] = plt.subplots(1,2,figsize=(10,20))
    ax1.imshow(imdata)
    for x in range(len(mask)):
        numbering = mpl_text.Text(x=alignlocs[0][x],y=alignlocs[1][x],text=str(x),color='r')
        ax1.add_artist(mask[x].bbox.as_artist(facecolor='none', edgecolor='white'))
        ax1.add_artist(numbering)
    #ax2.imshow(zeros)
    ax2.imshow(imdata)
    plt.show()

In [8]:
def cosineRule(masterStar,star1,star2):
    '''
    Calculate angle between three stars as masterStar as the centre
    
    Parameters:
        masterStar (lst): [x,y] image position of centre star
        star1 (lst): [x,y] image position of star1
        star2 (lst): [x,y] image position of star2
    
    Returns:
        CRounded (float): Angle between three input stars rounded to 1 decimal point
    '''
    a = ((masterStar[0]-star2[0])**2 + (masterStar[1]-star2[1])**2)**0.5
    b = ((star1[0]-masterStar[0])**2 + (star1[1]-masterStar[1])**2)**0.5
    c = (star1[0]-star2[0])**2 + (star1[1]-star2[1])**2
    C = np.arccos((a**2 + b**2 - c)/(2*a*b))
    CRounded = round(C,1)
    return CRounded

In [9]:
def findAllAnglesPerStar(table,starNo):
    '''
    Calculate all angles between a master stars and all other stars in a table
    
    Parameters:
        table (table): Astropy table of star positions
        starNo (int): Index of master star in table
    
    Returns:
        allAngles (lst): Nested lists containing all angles 
    '''
    
    masterStar = [table['X_IMAGE'][starNo],table['Y_IMAGE'][starNo]]
    allAngles = []
    for x in range(len(table)):
        if x != starNo:
            anglePerStar = []
            star1 = [table['X_IMAGE'][x],table['Y_IMAGE'][x]]
            for y in range(len(table)):
                if y != x and y!= starNo:
                    star2 = [table['X_IMAGE'][y],table['Y_IMAGE'][y]]
                    anglePerStar.append(cosineRule(masterStar,star1,star2))
            allAngles.append(anglePerStar)
    return allAngles

In [48]:
def distance(sources,object1,object2):
    """
    Find the distance between two objects in an image.

    :param sources: Table of object data
    :param object1: Index of first object in sources
    :param object2: Index of second object in sources
    :param imageNo: Image Number
    :return: Diagonal distance between the two objects
    """
    D = (((abs(sources['X_IMAGE'][object1]-sources['X_IMAGE'][object2])**2) + (abs(sources['Y_IMAGE'][object1]-sources['Y_IMAGE'][object2])**2))**0.5)
    Drounded = round(D,0)
    #print([round(x/5,-1)*5 for x in a])
    return Drounded


In [47]:
a = 111.111
print(round(a,0))

111.0


In [11]:
def referenceDistances(sources,samplestars,imageNo):
    """
    Calculate the diagonal distance between every object in samplestars.

    :param sources: Table of object data
    :param samplestars: List containing the indexs of stars
    :param imageNo: reference image number
    :return: List containing a list for each object, comprised of the diagonal distance between that object and every object in samplestars
    """
    #referencedistances = [[]]*len(samplestars)
    referencedistances = [[] for x in samplestars]
    for y in range(len(samplestars)):
        refd = []
        for x in range(len(samplestars)):
            refd.append(distance(sources,samplestars[y],samplestars[x],imageNo))
        referencedistances[y] = refd #.append(distance(sources,samplestars[y],samplestars[x],imageNo))
    
    #astroprov.provcall([sources,samplestars,imageNo],[referencedistances],"referenceDistances_PythonCode2PythonCode_SQ_tmpl.provn","referenceDistances")
    return referencedistances


In [12]:
def findFullDistancesPerObject(sources,objectNo):
    """
    Find the diagonal distances between every object in the image.

    :param sources: Table of object data
    :param imageno: Image number
    :return: Digonal distance between every object in the image
    """

    fulldistances = []
    for y in range(len(sources)):
        fulldistances.append(distance(sources,objectNo,y)) 
    
    #astroprov.provcall([sources,imageno],[fulldistances],"findFullDistances_PythonCode2PythonCode_SQ_tmpl.provn","findFullDistances")
    #fulldistances = [round(number/10,1) * 10 for number in fulldistances] # rounding can be changed by changing the mul/div
    return fulldistances

In [ ]:
# def matchAngles(allAnglesPerObject,allAngles):
#     '''
#     Match angle set from template image to that from target image.
    
#     Parameters:
#         allAnglesPerObject (lst): Angle set from template image.
#         allAngles (lst): Angle set from target image.
    
#     Returns:
#         translationDictionary (dict): Index of objects with matching angle sets between images.
#     '''
    
#     percentageForMatch = 0.9

#     # keep for now, inner join on an sql table might be better... ask Age about this?

#     translationDictionary = {}

#     for i,obj in enumerate(allAnglesPerObject): # loop over each object's angle set
#         oneObjectDict = {}
#         for j,angleSet in enumerate(obj): # loop over one angle set
#             matchLengths = []
#             for k,ogObj in enumerate(allAngles): # loop over master angle set
#                 result = all(elem in angleSet for elem in ogObj) # here is the problem-need to relax these constraints and change to any 
#                 #result = any(elem in angleSet for elem in ogObj) # here is the problem-need to relax these constraints and change to any 
                
#                 if result:
#                     if j not in oneObjectDict.items():
#                         oneObjectDict[j] = [k]
#                     else:
#                         oneObjectDict[j].append(k)
#         translationDictionary[i] = oneObjectDict
#     return translationDictionary # values are -1 of their true values - this is accounted for later

###########
# alternate match any condition, using all instead of any - too harsh for most cases so the below function is preferred

In [65]:
def matchAngles(allAnglesPerObject,allAngles):
    '''
    Match angle set from template image to that from target image.
    
    Parameters:
        allAnglesPerObject (lst): Angle set from template image.
        allAngles (lst): Angle set from target image.
    
    Returns:
        translationDictionary (dict): Index of objects with matching angle sets between images.
    '''
    percentageForMatch = 0.9

    # keep for now, inner join on an sql table might be better... ask Age about this?

    translationDictionary = {}

    for i,obj in enumerate(allAnglesPerObject): # loop over each object's angle set
        oneObjectDict = {}
        for j,angleSet in enumerate(obj): # loop over one angle set
            matchLengths = []
            for k,ogObj in enumerate(allAngles): # loop over master angle set
                #result = all(elem in angleSet for elem in ogObj) # here is the problem-need to relax these constraints and change to any 
                result = any(elem in angleSet for elem in ogObj) # here is the problem-need to relax these constraints and change to any 
                
                if result:
                    oneObjectDict[j] = k
        translationDictionary[i] = oneObjectDict
    return translationDictionary # values are -1 of their true values - this is accounted for later

In [68]:
def confirmAngleMatchWithDistances(translationDictionary,refTab,referenceDistances,indexing):
    '''
    Match the distances to potential matches found thorugh angles.
    
    Parameters:
        translationDictionary (dict): Indexes of potential matches across two images.
        refTab (astropy table): Table of reference objects.
        referenceDistances (lst): Distances between objects in refTab.
        indexing (list): List of object indexes, correcting for astropy tables starting with an index of 1.
        
    Returns:
        trueMatchDict (dict): Dictionary of indexes corresponding to matched objects.
    '''
    # note, this function does not included a distance tollerance and should only be used when the reference distances have been rounded
    trueMatchDict = {}
    potentialMatches = []
    potentialRef = []
    for key,value in translationDictionary.items():
        oneMatch = []
        oneRef = []
        if len(value) !=0:
            for imgNo,sampleNo in value.items():
                objectDistance = distance(refTab,key,imgNo+1)
                if objectDistance == referenceDistances[indexing.index(sampleNo)]:
                    oneMatch.append(imgNo+1)
                    oneRef.append(sampleNo)
        potentialMatches.append(oneMatch)
        potentialRef.append(oneRef)

    listLengths = [len(x) for x in potentialMatches]
    bestMatch = max(listLengths)
    bestMatch = [i for i,j in enumerate(listLengths) if j==bestMatch]
    if len(bestMatch) > 1:
        print('Currently {} potentail candidates, choosing the first'.format(len(bestMatch)))
    trueMatchDict[0] = bestMatch[0]
    for x in range(len(potentialMatches[bestMatch[0]])):
        trueMatchDict[potentialRef[bestMatch[0]][x]] = potentialMatches[bestMatch[0]][x]
    return trueMatchDict

In [82]:
def confirmAngleMatchWithDistances(translationDictionary,refTab,referenceDistances,indexing,closeness=10):
    '''
    Match the distances to potential matches found thorugh angles with a tollerance.
    
    Parameters:
        translationDictionary (dict): Indexes of potential matches across two images.
        refTab (astropy table): Table of reference objects.
        referenceDistances (lst): Distances between objects in refTab.
        indexing (list): List of object indexes, correcting for astropy tables starting with an index of 1.
        closeness (int): Distance tollerance for two objects to be considered the same.
        
    Returns:
        trueMatchDict (dict): Dictionary of indexes corresponding to matched objects.
    '''
    trueMatchDict = {}
    potentialMatches = []
    potentialRef = []
    for key,value in translationDictionary.items():
        oneMatch = []
        oneRef = []
        if len(value) !=0:
            for imgNo,sampleNo in value.items():
                oneMatchSlim = []
                oneRefSlim = []
                for number in sampleNo:
                    objectDistance = distance(refTab,key,imgNo)
                    refDist = referenceDistances[indexing.index(number)]
                    if objectDistance > (refDist-closeness) and objectDistance < (refDist+closeness):
                        oneMatchSlim.append(imgNo)
                        oneRefSlim.append(number)
                if len(oneMatchSlim) > 1:
                    print('Object {} Matches for candidates: {}, choosing {}'.format(imgNo,oneMatchSlim,oneMatchSlim[0]))
                    oneMatch.append(oneMatchSlim[0])
                    oneRefSlim.append(oneRefSlim[0])
                elif len(oneMatchSlim) > 0:
                    oneMatch.append(oneMatchSlim[0])
                    oneRefSlim.append(oneRefSlim[0])

        potentialMatches.append(oneMatch)
        potentialRef.append(oneRef)
    
    listLengths = [len(x) for x in potentialMatches]
    bestMatch = max(listLengths)
    bestMatch = [i for i,j in enumerate(listLengths) if j==bestMatch]
    if len(bestMatch) > 1:
        print('Currently {} potentail candidates, choosing the first'.format(len(bestMatch)))
    trueMatchDict[0] = bestMatch[0]
    print(bestMatch)
    print(potentialMatches)
    for x in range(len(potentialMatches[bestMatch[0]])):
        trueMatchDict[potentialRef[bestMatch[0]][x]] = potentialMatches[bestMatch[0]][x]
    return trueMatchDict

In [84]:
def molePositions2Tab(molePositions):
    '''
    Create dataframe of mole x and y positions with upper and lower bounds for easy cross-matching.
    
    Parameters:
        molePositions (str): Name of DS9 region file containing mole positions.
        
    Returns:
        refPandasNew (pandas dataframe): Dataframe containing mole locations and upper/lower bounds.
    '''
    
    with open(molePositions,'r') as f:
        data = f.readlines()

    ycoord = []
    xcoord = []
    starLocations = []
    for x in data:
        if 'circle(' in x:
            locator1 = '('
            locator2 = ','
            loc1 = x.index(locator1) +1
            #print(x)
            loc2 = x[loc1:].index(locator2) + loc1 +1
            loc3 = x[loc2:].index(locator2) + loc2
            xcoord.append(x[loc1:loc2-1])
            ycoord.append(x[loc2:loc3])
            #starLocations.append([float(xcoord),float(ycoord)])
    tollerance = 10

    xUpper = [float(x) + tollerance for x in xcoord]
    xLower = [float(x) - tollerance for x in xcoord]
    yUpper = [float(x) + tollerance for x in ycoord]
    yLower = [float(x) - tollerance for x in ycoord]

    refDict = {'index':range(len(xcoord)),'xcoord':xcoord,'ycoord':ycoord,'xUpper':xUpper,'xLower':xLower,
              'yUpper':yUpper,'yLower':yLower}
    refPandasNew = pd.DataFrame(data=refDict)
    return refPandasNew

In [17]:
def assessQuality(output,sqlQuery,refPandasNew):
    '''
    Match tables of objects found between two images to determine if the matching was good.
    
    Parameters:
        output (astropy table): Table of objects found in original image.
        sqlQuery (str): SQL command join the two tables.
        refPandasNew (pandas dataframe): Dataframe of objects found via the matching algorithm.
    
    Returns:
        newdf (pandas dataframe): Matched objects the two input sources.
    '''
    pandasTable = Table.to_pandas(output)
    newdf = ps.sqldf(sqlQuery,locals())
    objectIndexes = newdf['index'].tolist()
    oldIndexes = refPandasNew['index'].tolist()
    objectsNotFound = [x for x in oldIndexes if x not in objectIndexes]
    if len(objectsNotFound) != 0:
        print('Warning: Reference object(s) with indexes {} not found in original table, Excluding them from calculation'.format(objectsNotFound))
    return newdf

In [18]:
sqlcode = '''
SELECT *
FROM pandasTable
JOIN refPandasNew
ON pandasTable.X_IMAGE < refPandasNew.xUpper
AND pandasTable.X_IMAGE > refPandasNew.xLower
AND pandasTable.Y_IMAGE < refPandasNew.yUpper
AND pandasTable.Y_IMAGE > refPandasNew.yLower
'''
# sql code for matching objects, currently querying through pandas dataframes which is horrible for memory and
# timing. To improve this, put all of this processing into queries over Apache Spark dataframes instead. 

In [85]:
refPandasNew = molePositions2Tab('/home/mj1e16/moleGazer/Photos/F/fitsImages/16bit/inputRegions29018.reg')

In [86]:
targetMoles = assessQuality(refTab,sqlcode,refPandasNew)

In [87]:
targetMolesAstropy = Table.from_pandas(targetMoles)

In [96]:
ascii.write(targetMolesAstropy,'./testSampleTable.tab',format='csv')

In [95]:
targetMolesAstropy['NUMBER'].tolist()

[8, 11, 15, 16, 17, 19, 21, 28, 38, 41, 42, 45, 48, 53]

In [89]:
#testSources = findTestSources([refTab],0,10000,0,10000,0)

In [90]:
# targetMoles = [44,37,35,34,40,41,20,16,15,14,10]
# targetMolesReal = [x+1 for x in targetMoles] 
# fullListofObjects = range(len(refTab))
# toBeRemoved = [x for x in fullListofObjects if x not in targetMoles]
# #print(len(fullListofObjects),len(toBeRemoved),len(testSources))
# refTab.remove_rows(toBeRemoved)
# sampleStars = refTab
# refTab = Table.read('/home/mj1e16/moleGazer/Photos/F/fitsImages/simMolePos/29018InvertedAverageCropped166.44444444444_21.8421052632_gauss_5.0_9x9.conv.cat',format='ascii.sextractor')

In [91]:
allAngles = findAllAnglesPerStar(targetMolesAstropy,0) # all angles between master and sample stars
#allAngles = findAllAnglesPerStar(sampleStars,0) # all angles between master and sample stars

allAnglesPerObject = [] # nested list - each list represents one star chosen as the master star and the values within it are the angles between each other star
for x in range(len(refTab)):
    allAnglesPerObject.append(findAllAnglesPerStar(refTab,x))


In [92]:
translationDict = matchAngles(allAnglesPerObject,allAngles)

In [93]:
# referenceDistances = []
# indexing = []
# for x in range(1,len(sampleStars)):
#     referenceDistances.append(distance(sampleStars,0,x))
#     indexing.append(x-1)

referenceDistances = []
indexing = []
for x in range(len(targetMolesAstropy)):
    referenceDistances.append(distance(targetMolesAstropy,0,x))
    indexing.append(x)

In [94]:
newtrueMatchDict = confirmAngleMatchWithDistances(translationDict,refTab,referenceDistances,indexing,closeness=50)

[10]
[[28, 38, 45], [], [21, 23, 26], [32, 47, 48], [32, 48], [34, 35, 37, 44, 51], [10, 39, 40, 41, 42, 43], [32, 43, 46, 47, 48], [15], [], [9, 10, 33, 34, 35, 36, 44], [], [], [24], [], [], [], [], [], [], [20, 24], [21], [], [2], [24], [], [], [12], [0], [], [], [47], [3, 4, 7, 41], [10], [5, 10], [5, 10], [5, 10, 37], [5], [0, 23], [3, 6, 26, 30], [6, 21, 37, 43], [6, 37], [0, 42], [], [5, 10, 51], [0], [3, 29, 36], [36], [3, 4, 7, 50], [], [], [5], [0]]


IndexError: list index out of range

In [63]:
newtrueMatchDict

{0: 2, 10: 29, 11: 48}

In [66]:
translationDict[0]

{7: 0,
 8: 11,
 9: 9,
 10: 1,
 11: 1,
 12: 3,
 13: 10,
 14: 11,
 15: 11,
 16: 11,
 17: 1,
 18: 10,
 19: 3,
 20: 11,
 21: 11,
 22: 3,
 23: 10,
 24: 11,
 25: 11,
 26: 9,
 27: 10,
 28: 3,
 29: 3,
 30: 11,
 31: 9,
 32: 10,
 33: 10,
 34: 10,
 35: 3,
 36: 11,
 37: 11,
 38: 10,
 39: 11,
 40: 10,
 41: 10,
 42: 9,
 43: 9,
 44: 3,
 45: 11,
 46: 11,
 47: 9,
 48: 3,
 49: 3,
 50: 10,
 51: 11}